# Object Detection with YOLOV5

## Install and Import Dependencies

In [1]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
#!git clone https://github.com/ultralytics/yolov5

In [3]:
import torch
from matplotlib import pyplot as plt  
import numpy as np
import cv2 as cv
import uuid
import os
import time
import json
import pandas as pd
import numpy as np

## Twilio Setup

In [4]:
with open(r"C:\Users\kdelfino\.secret\twilio_creds.json", "r") as f:
    login = json.load(f)

# Display loaded dict 
login.keys()

dict_keys(['client-id', 'Auth-Token'])

In [5]:
from twilio.rest import Client

account_sid = login['client-id']
auth_token = login['Auth-Token']
client = Client(account_sid, auth_token)



## Training The Model

In [6]:
# example making new folder with os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['images', 'labels', 'labels.cache', 'labels.cache.npy', 'video']

In [7]:
IMAGES_PATH = r"C:\Users\kdelfino\Documents\GitHub\Object_Detection\object_detection\Data\images"
labels = ['kim','bottle', 'drinking']
img_count = 10

In [8]:
IMAGES_PATH

'C:\\Users\\kdelfino\\Documents\\GitHub\\Object_Detection\\object_detection\\Data\\images'

In [9]:
cap = cv.VideoCapture(1)

# Loop through labels
for label in labels:
    print(f'Collecting images for {label}')
    time.sleep(5)

    # Loop through image range
    for count in range(img_count):
        print(f'Collecting images for {label} #{count}')

        # Webcam feed
        ret, frame = cap.read()

        # Naming out image path
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')

        # Writes out image to file
        cv.imwrite(imgname, frame)
        
        # Render to the screen
        cv.imshow('Image Collection', frame)

        # 2 sec delay between captures
        time.sleep(2)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()
        


In [10]:
# for label in labels:
#     print('Collecting images for {}'.format(label))
#     for img_num in range(img_count):
#         print('Collecting images for {}, image number {}'.format(label, img_num))
#         imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
#         print(imgname)   

In [11]:
#!git clone https://github.com/tzutalin/labelImg

In [12]:
# !pip install pyqt5 lxml --upgrade
# !cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

In [13]:
# !cd yolov5 && python train.py --img 320 --batch 16 --epochs 500 --data dataset.yaml --weights yolov5s.pt --workers 2

In [14]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'C:\Users\kdelfino\Documents\GitHub\Object_Detection\object_detection\yolov5\runs\train\exp6\weights\last.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\kdelfino/.cache\torch\hub\master.zip
YOLOv5  2023-8-15 Python-3.9.15 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7058671 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [15]:
vid = r"Data\video\WIN_20230814_23_44_04_Pro.mp4"

In [25]:
cap = cv.VideoCapture(vid)
# while cap.isOpened():
waiting_sms = True
while True:
    ret, frame = cap.read()

    results = model(frame)

    cv.imshow('YOLO', np.squeeze(results.render()))
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
    if  results.pandas().xyxy[0].name.to_string().split(' ')[-1] == 'drinking':
            if waiting_sms:
                message = client.messages.create(
                from_= #'TWIOLIO NUMBER',
                body = 'Great work rehydrating! Next reminder will be in 30mins! ',
                to= #'PHONE NUMBER'
                )

                print(message.sid)

                waiting_sms = False

    
cap.release()
cv.destroyAllWindows()


SM10f09fb77936fe2127a43eb9d36d7940


In [17]:
img = r"Data\images\drinking.e208e343-3b26-11ee-8280-701ab828e848.jpg"

In [18]:
results = model(img)

In [19]:
print(results)

image 1/1: 480x640 1 drinking
Speed: 11.5ms pre-process, 108.9ms inference, 1.0ms NMS per image at shape (1, 3, 480, 640)
